In [38]:
!pip install fastaparser

Defaulting to user installation because normal site-packages is not writeable


In [47]:
import fastaparser as fst
import pandas as pd

In [62]:
contig_path = "Poil_contig.fa"
scaffold_path = "Poil_scaffold.fa"
gapClosed_path = "Poil_gapClosed.fa"

In [127]:
# Парсим fasta в Pandas Dataframe
def parser(path):
    with open('trim/'+path, "r") as file:
        seq_parser = fst.Reader(file, parse_method='quick')
        df = pd.DataFrame(columns=["Sequence", "Length"])
        for seq in seq_parser:
            df.loc[seq.header, 'Sequence'] = seq.sequence
            df.loc[seq.header, 'Length'] = len(seq.sequence)
    return df.sort_values(by="Length", ascending=False)

def general_stats(df):
    print("Number of sequences:", len(df))
    print("Total length:", df.Length.sum())
    print("The size of the longest sequence:", df.Length[0])
    i = 1; len_sum = df.Length[0]
    while len_sum < df.Length.sum() / 2:
        len_sum += df.Length[i]
        i += 1
    print("N50:", df.Length[i])
    
def gaps(seq):
    print("Number of gaps:", len(re.findall("N+", seq)))
    print("The gaps themselves: ", re.findall("N+", seq))
    print("Total length of gaps:", seq.count("N"))

In [89]:
contigs = parser(contig_path)
scaffold = parser(scaffold_path)
gapClosed = parser(gapClosed_path)

In [129]:
print("\tStatistics for contigs"); general_stats(contigs)
print("\n\tStatistics for scaffolds"); general_stats(scaffold); gaps(scaffold.Sequence[0])
print("\n\tStatistics for gapClosed"); general_stats(gapClosed); gaps(gapClosed.Sequence[0])

	Statistics for contigs
Number of sequences: 609
Total length: 3925016
The size of the longest sequence: 179307
N50: 47611

	Statistics for scaffolds
Number of sequences: 95
Total length: 3870305
The size of the longest sequence: 383603
N50: 169882
Number of gaps: 3
The gaps themselves:  ['N', 'NNNNNNNNNNNNNN', 'NNNNNNNNNNNNNNNNNNNNNNNNN']
Total length of gaps: 40

	Statistics for gapClosed
Number of sequences: 95
Total length: 3870322
The size of the longest sequence: 383574
N50: 169882
Number of gaps: 0
The gaps themselves:  []
Total length of gaps: 0


In [141]:
# Сохраняем самый длинный скаффолд в fasta-файл
with open(r"trim/longest.fasta", 'w') as fasta_file:
    writer = fst.Writer(fasta_file)
    writer.writefasta((gapClosed.index[0], gapClosed.Sequence[0]))